In [1]:
import numpy as np
import torch
from functions import load_test_data
from pickle import load

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
#3flavor_poisson, nsi_poisson
learn_target = 'nsi_poisson'
data_index = 1

data_all = load_test_data(learn_target, data_index)

scaler = load(open('nf/' + learn_target + '/scaler.pkl', 'rb'))
data_all = scaler.transform(data_all)

In [3]:
index = 32
flow = torch.load('nf/' + learn_target + '/{}.pt'.format(index))

In [4]:
if learn_target == '3flavor_poisson':
    n_sample = 1000000
    IO_samples = flow.sample(num_samples=n_sample,
                context=torch.tensor(np.array([data_all[1]]), dtype=torch.float32).to(device)).cpu().detach().numpy().reshape(-1, 4)
    NO_samples = flow.sample(num_samples=n_sample,
                context=torch.tensor(np.array([data_all[0]]), dtype=torch.float32).to(device)).cpu().detach().numpy().reshape(-1, 4)
    np.savez('./IO_contour/' + learn_target + '/nf.npz', IO_samples)
    np.savez('./NO_contour/' + learn_target + '/nf.npz', NO_samples)
else:
    n_sample = 100000
    prediction = flow.sample(num_samples=n_sample,
                context=torch.tensor(np.array(data_all), dtype=torch.float32).to(device)).cpu().detach().numpy().reshape(-1, 7)
    np.savez('contour/' + learn_target +  '_{}/nf.npz'.format(data_index), prediction)